# Final Project

**Họ và tên thành viên 1:** Trần Tuấn Đạt

**MSSV:** 18120313

**Họ và tên thành viên 2:** Bùi Trọng Hoàng

**MSSV:** 18120537

---

## Thu thập dữ liệu

---

## Khám phá dữ liệu

---

## Đưa ra câu hỏi để trả lời

---

### Tiền xử lý

---

### Phân tích dữ liệu để trả lời